In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import dlib
import re

In [3]:
import cv2

# Define a few variables

In [4]:
basedir = './dataset_AMLS_22-23/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Define a set of functions to be used to extract the features 

In [5]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [6]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [7]:
def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

In [15]:
def resize_image(img_array):
    pil_image = Image.fromarray(img_array)
    resized_image = pil_image.resize((5, 5))
    return np.array(resized_image)

In [16]:
# This function will return the RBG values of the cropped images (we only care about the eye, not the whole face)
def get_eye_2(image_file):
    image = Image.open(f'dataset_AMLS_22-23/cartoon_set/img/{image_file}')
    image_array = np.array(image)
    features,_ = run_dlib_shape(image_array)
    eye_features = [features[37], features[38], features[40], features[41]]
    horizontal_pixels = [element[0] for element in eye_features]
    vertical_pixels = [element[1] for element in eye_features] 
    iris_crop = [(min(horizontal_pixels), max(horizontal_pixels)), (min(vertical_pixels), max(vertical_pixels))]
    image_array = image_array[iris_crop[1][0]:iris_crop[1][1], iris_crop[0][0]:iris_crop[0][1], :]
    resized_image_arr = resize_image(image_array)
    """
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    right_eye = img_array[250:277, 190:227, :]
    print(f'Completed eye {iteration}')
    return right_eye.reshape((1, 2997))
    """
    return resized_image_arr[:, :, 0:3].reshape((1, 75))/255

# Create a dataset comprised of the iris images

In [10]:
cartoon_df = pd.read_csv('dataset_AMLS_22-23/cartoon_set/labels.csv', sep='\t')

In [11]:
cartoon_df.drop(cartoon_df.columns[0], axis=1, inplace=True)

In [12]:
cartoon_df

,eye_color,face_shape,file_name
0,1,4,0.png
1,2,4,1.png
2,2,3,2.png
3,2,0,3.png
4,0,2,4.png
...,...,...,...
9995,3,2,9995.png
9996,0,3,9996.png
9997,1,2,9997.png
9998,0,2,9998.png


In [18]:
# Create a list containing the RGB values of the cropped images
iris_list = []
labels_list = []
for i in range(10000):
    print(f'Iteration number: {i} out of 10000')
    try:
        iris_array = get_eye_2(f'{i}.png')
        iris_list.append(iris_array)
        labels_list.append(int(cartoon_df[cartoon_df['file_name'] == f'{i}.png']['eye_color']))
    except:
        continue

Iteration number: 0 out of 10000
Iteration number: 1 out of 10000
Iteration number: 2 out of 10000
Iteration number: 3 out of 10000
Iteration number: 4 out of 10000
Iteration number: 5 out of 10000
Iteration number: 6 out of 10000
Iteration number: 7 out of 10000
Iteration number: 8 out of 10000
Iteration number: 9 out of 10000
Iteration number: 10 out of 10000
Iteration number: 11 out of 10000
Iteration number: 12 out of 10000
Iteration number: 13 out of 10000
Iteration number: 14 out of 10000
Iteration number: 15 out of 10000
Iteration number: 16 out of 10000
Iteration number: 17 out of 10000
Iteration number: 18 out of 10000
Iteration number: 19 out of 10000
Iteration number: 20 out of 10000
Iteration number: 21 out of 10000
Iteration number: 22 out of 10000
Iteration number: 23 out of 10000
Iteration number: 24 out of 10000
Iteration number: 25 out of 10000
Iteration number: 26 out of 10000
Iteration number: 27 out of 10000
Iteration number: 28 out of 10000
Iteration number: 29 out

Iteration number: 238 out of 10000
Iteration number: 239 out of 10000
Iteration number: 240 out of 10000
Iteration number: 241 out of 10000
Iteration number: 242 out of 10000
Iteration number: 243 out of 10000
Iteration number: 244 out of 10000
Iteration number: 245 out of 10000
Iteration number: 246 out of 10000
Iteration number: 247 out of 10000
Iteration number: 248 out of 10000
Iteration number: 249 out of 10000
Iteration number: 250 out of 10000
Iteration number: 251 out of 10000
Iteration number: 252 out of 10000
Iteration number: 253 out of 10000
Iteration number: 254 out of 10000
Iteration number: 255 out of 10000
Iteration number: 256 out of 10000
Iteration number: 257 out of 10000
Iteration number: 258 out of 10000
Iteration number: 259 out of 10000
Iteration number: 260 out of 10000
Iteration number: 261 out of 10000
Iteration number: 262 out of 10000
Iteration number: 263 out of 10000
Iteration number: 264 out of 10000
Iteration number: 265 out of 10000
Iteration number: 26

Iteration number: 474 out of 10000
Iteration number: 475 out of 10000
Iteration number: 476 out of 10000
Iteration number: 477 out of 10000
Iteration number: 478 out of 10000
Iteration number: 479 out of 10000
Iteration number: 480 out of 10000
Iteration number: 481 out of 10000
Iteration number: 482 out of 10000
Iteration number: 483 out of 10000
Iteration number: 484 out of 10000
Iteration number: 485 out of 10000
Iteration number: 486 out of 10000
Iteration number: 487 out of 10000
Iteration number: 488 out of 10000
Iteration number: 489 out of 10000
Iteration number: 490 out of 10000
Iteration number: 491 out of 10000
Iteration number: 492 out of 10000
Iteration number: 493 out of 10000
Iteration number: 494 out of 10000
Iteration number: 495 out of 10000
Iteration number: 496 out of 10000
Iteration number: 497 out of 10000
Iteration number: 498 out of 10000
Iteration number: 499 out of 10000
Iteration number: 500 out of 10000
Iteration number: 501 out of 10000
Iteration number: 50

Iteration number: 709 out of 10000
Iteration number: 710 out of 10000
Iteration number: 711 out of 10000
Iteration number: 712 out of 10000
Iteration number: 713 out of 10000
Iteration number: 714 out of 10000
Iteration number: 715 out of 10000
Iteration number: 716 out of 10000
Iteration number: 717 out of 10000
Iteration number: 718 out of 10000
Iteration number: 719 out of 10000
Iteration number: 720 out of 10000
Iteration number: 721 out of 10000
Iteration number: 722 out of 10000
Iteration number: 723 out of 10000
Iteration number: 724 out of 10000
Iteration number: 725 out of 10000
Iteration number: 726 out of 10000
Iteration number: 727 out of 10000
Iteration number: 728 out of 10000
Iteration number: 729 out of 10000
Iteration number: 730 out of 10000
Iteration number: 731 out of 10000
Iteration number: 732 out of 10000
Iteration number: 733 out of 10000
Iteration number: 734 out of 10000
Iteration number: 735 out of 10000
Iteration number: 736 out of 10000
Iteration number: 73

Iteration number: 946 out of 10000
Iteration number: 947 out of 10000
Iteration number: 948 out of 10000
Iteration number: 949 out of 10000
Iteration number: 950 out of 10000
Iteration number: 951 out of 10000
Iteration number: 952 out of 10000
Iteration number: 953 out of 10000
Iteration number: 954 out of 10000
Iteration number: 955 out of 10000
Iteration number: 956 out of 10000
Iteration number: 957 out of 10000
Iteration number: 958 out of 10000
Iteration number: 959 out of 10000
Iteration number: 960 out of 10000
Iteration number: 961 out of 10000
Iteration number: 962 out of 10000
Iteration number: 963 out of 10000
Iteration number: 964 out of 10000
Iteration number: 965 out of 10000
Iteration number: 966 out of 10000
Iteration number: 967 out of 10000
Iteration number: 968 out of 10000
Iteration number: 969 out of 10000
Iteration number: 970 out of 10000
Iteration number: 971 out of 10000
Iteration number: 972 out of 10000
Iteration number: 973 out of 10000
Iteration number: 97

...

Iteration number: 8968 out of 10000
Iteration number: 8969 out of 10000
Iteration number: 8970 out of 10000
Iteration number: 8971 out of 10000
Iteration number: 8972 out of 10000
Iteration number: 8973 out of 10000
Iteration number: 8974 out of 10000
Iteration number: 8975 out of 10000
Iteration number: 8976 out of 10000
Iteration number: 8977 out of 10000
Iteration number: 8978 out of 10000
Iteration number: 8979 out of 10000
Iteration number: 8980 out of 10000
Iteration number: 8981 out of 10000
Iteration number: 8982 out of 10000
Iteration number: 8983 out of 10000
Iteration number: 8984 out of 10000
Iteration number: 8985 out of 10000
Iteration number: 8986 out of 10000
Iteration number: 8987 out of 10000
Iteration number: 8988 out of 10000
Iteration number: 8989 out of 10000
Iteration number: 8990 out of 10000
Iteration number: 8991 out of 10000
Iteration number: 8992 out of 10000
Iteration number: 8993 out of 10000
Iteration number: 8994 out of 10000
Iteration number: 8995 out o

Iteration number: 9197 out of 10000
Iteration number: 9198 out of 10000
Iteration number: 9199 out of 10000
Iteration number: 9200 out of 10000
Iteration number: 9201 out of 10000
Iteration number: 9202 out of 10000
Iteration number: 9203 out of 10000
Iteration number: 9204 out of 10000
Iteration number: 9205 out of 10000
Iteration number: 9206 out of 10000
Iteration number: 9207 out of 10000
Iteration number: 9208 out of 10000
Iteration number: 9209 out of 10000
Iteration number: 9210 out of 10000
Iteration number: 9211 out of 10000
Iteration number: 9212 out of 10000
Iteration number: 9213 out of 10000
Iteration number: 9214 out of 10000
Iteration number: 9215 out of 10000
Iteration number: 9216 out of 10000
Iteration number: 9217 out of 10000
Iteration number: 9218 out of 10000
Iteration number: 9219 out of 10000
Iteration number: 9220 out of 10000
Iteration number: 9221 out of 10000
Iteration number: 9222 out of 10000
Iteration number: 9223 out of 10000
Iteration number: 9224 out o

Iteration number: 9427 out of 10000
Iteration number: 9428 out of 10000
Iteration number: 9429 out of 10000
Iteration number: 9430 out of 10000
Iteration number: 9431 out of 10000
Iteration number: 9432 out of 10000
Iteration number: 9433 out of 10000
Iteration number: 9434 out of 10000
Iteration number: 9435 out of 10000
Iteration number: 9436 out of 10000
Iteration number: 9437 out of 10000
Iteration number: 9438 out of 10000
Iteration number: 9439 out of 10000
Iteration number: 9440 out of 10000
Iteration number: 9441 out of 10000
Iteration number: 9442 out of 10000
Iteration number: 9443 out of 10000
Iteration number: 9444 out of 10000
Iteration number: 9445 out of 10000
Iteration number: 9446 out of 10000
Iteration number: 9447 out of 10000
Iteration number: 9448 out of 10000
Iteration number: 9449 out of 10000
Iteration number: 9450 out of 10000
Iteration number: 9451 out of 10000
Iteration number: 9452 out of 10000
Iteration number: 9453 out of 10000
Iteration number: 9454 out o

Iteration number: 9658 out of 10000
Iteration number: 9659 out of 10000
Iteration number: 9660 out of 10000
Iteration number: 9661 out of 10000
Iteration number: 9662 out of 10000
Iteration number: 9663 out of 10000
Iteration number: 9664 out of 10000
Iteration number: 9665 out of 10000
Iteration number: 9666 out of 10000
Iteration number: 9667 out of 10000
Iteration number: 9668 out of 10000
Iteration number: 9669 out of 10000
Iteration number: 9670 out of 10000
Iteration number: 9671 out of 10000
Iteration number: 9672 out of 10000
Iteration number: 9673 out of 10000
Iteration number: 9674 out of 10000
Iteration number: 9675 out of 10000
Iteration number: 9676 out of 10000
Iteration number: 9677 out of 10000
Iteration number: 9678 out of 10000
Iteration number: 9679 out of 10000
Iteration number: 9680 out of 10000
Iteration number: 9681 out of 10000
Iteration number: 9682 out of 10000
Iteration number: 9683 out of 10000
Iteration number: 9684 out of 10000
Iteration number: 9685 out o

Iteration number: 9886 out of 10000
Iteration number: 9887 out of 10000
Iteration number: 9888 out of 10000
Iteration number: 9889 out of 10000
Iteration number: 9890 out of 10000
Iteration number: 9891 out of 10000
Iteration number: 9892 out of 10000
Iteration number: 9893 out of 10000
Iteration number: 9894 out of 10000
Iteration number: 9895 out of 10000
Iteration number: 9896 out of 10000
Iteration number: 9897 out of 10000
Iteration number: 9898 out of 10000
Iteration number: 9899 out of 10000
Iteration number: 9900 out of 10000
Iteration number: 9901 out of 10000
Iteration number: 9902 out of 10000
Iteration number: 9903 out of 10000
Iteration number: 9904 out of 10000
Iteration number: 9905 out of 10000
Iteration number: 9906 out of 10000
Iteration number: 9907 out of 10000
Iteration number: 9908 out of 10000
Iteration number: 9909 out of 10000
Iteration number: 9910 out of 10000
Iteration number: 9911 out of 10000
Iteration number: 9912 out of 10000
Iteration number: 9913 out o

Once we have obtained both lists, they will be stored so that we don't need to generate the dataset every time we run the script.

In [22]:
%store iris_list

Stored 'iris_list' (list)


In [23]:
%store labels_list

Stored 'labels_list' (list)
